In [1]:
from pathlib import Path

import numpy as np
from scipy import stats
from shapely.geometry import Point
import geopandas

In [2]:
# https://math.stackexchange.com/q/444700/593484
# See alternative method 1: https://corysimon.github.io/articles/uniformdistn-on-sphere/
NUM_WORLD_LOCATIONS = 37067
# in miles:
MEAN_RADIUS_OF_EARTH = 3958.7613
DATA_PATH = Path("../data/")


In [9]:
def cartesian_to_gcs(v):
    """Change the basis used to represent
    a *unit* vector in R^3 (3-dimensional Euclidean space)
    from the standard basis for R^3 into a basis
    for a geographical coordinate system (GCS).  Output is
    in degrees. Only the latitude and longitude are returned
    (in that order).

    Args:
        v: numpy.ndarray

    Returns:
        numpy.ndarray
    """
    # https://en.wikipedia.org/wiki/Spherical_coordinate_system#In_geography
    x = v[0]
    y = v[1]
    z = v[2]
    # Assume v has L-2 norm of 1.
    r = 1.0
    # https://stackoverflow.com/questions/50906886/convert-spherical-to-geographical-coordinate-system
    # Calculate the latitude
    theta = np.arccos(z / r) * 180.0 / np.pi
    latitude = theta - 90.0
    # Calculate the longitude
    phi = np.sign(y) * np.arccos(x / np.sqrt(x**2 + y**2)) * 180.0 / np.pi
    longitude = phi
    return np.array([latitude, longitude])

def get_uniformly_distributed_points_on_unit_sphere(n:int):
    """The points are returned as an numpy.ndarray with
    latitudes in the 0th row and longitude in the 1st row.
    Each column represents a different point.

    References:
    https://math.stackexchange.com/q/444700/593484
    See alternative method 1: https://corysimon.github.io/articles/uniformdistn-on-sphere/
    https://gis.stackexchange.com/a/356502
    """
    # The user desires n points.  However, it is hard to generate
    # exactly n points in a numerically stable way.  Therefore,
    # we try to generate int(n * n_safety) points.
    vecs_on_sphere = np.empty(shape=(3, n))
    n_safety = 1.01
    num_points_generated = 0

    while num_points_generated < n:
        # Generate int(n * n_safety) vectors.
        # Store these vectors as columns in a matrix V
        # for further manipulation.
        # Only take those columns with a large enough norm.
        V = stats.norm.rvs(size=(3, int(n * n_safety)))
        norms_of_V_cols = np.linalg.norm(x=V, ord=2, axis=0, keepdims=False)
   
        # https://stackoverflow.com/a/49763264/8423001
        num_points_generated = np.count_nonzero(norms_of_V_cols > 0.0001)
        if num_points_generated < n:
            n_safety = n_safety + 0.01
            # continue with next iteration of loop
            continue
        # We found the number of vectors required by n
        # that meet our numerical requirement of being long enough.
        # The vectors can now
        # be stretched so that they land on the unit-sphere.
        vecs_on_sphere = V[:, norms_of_V_cols > 0.0001][:, 0:n] / norms_of_V_cols[norms_of_V_cols > 0.0001][0:n]

    # We don't care about returning the radius.
    return np.apply_along_axis(
        func1d=cartesian_to_gcs,
        axis=0,
        arr=vecs_on_sphere
    )


# Test Functions

In [13]:
get_uniformly_distributed_points_on_unit_sphere(13)

array([[  14.24760034,  -64.72290427,   28.98287275,   28.09099116,
          37.03484059,  -84.43900495,   33.27115248,  -44.38430096,
         -26.6123818 ,   -2.99001652,  -51.13429991,  -13.46166205,
         -20.75404127],
       [ 112.13170506,  160.91339877,  -13.9724902 ,   36.78301105,
         -76.82078208,  -44.62191136,  -55.31522664, -143.63153923,
          11.70091987,   76.9691995 , -157.78916042,  -23.28613266,
         -52.09057287]])

In [ ]:
np.linalg.norm(x=np.array([[1, 3, 2], [1, 4, 2]]), ord=2, axis=0, keepdims=False) > 1.5

In [ ]:

vecs_on_sphere = np.empty(shape=(3, NUM_WORLD_LOCATIONS))
# Loop as many times as we want to get a vector
# on the unit-sphere.
for i in range(NUM_WORLD_LOCATIONS):
    # Try to get a vector long enough so
    # that it is easy to rescale it
    # to land on the unit-sphere.
    
    # Before loop
    norm_of_v = 0
    while (norm_of_v < 0.0001):
        # We have to try getting more numerically
        # stable random normal deviates.
        x = stats.norm.rvs(size=1).item()
        y = stats.norm.rvs(size=1).item()
        z = stats.norm.rvs(size=1).item()
        
        norm_of_v = np.sqrt(x**2 + y**2 + z**2)

    # We found a long enough vector that can now
    # be stretched so that it lands on the unit-sphere
    # without any numerical problems.
    vecs_on_sphere[0, i] = x / norm_of_v
    vecs_on_sphere[1, i] = y / norm_of_v
    vecs_on_sphere[2, i] = z / norm_of_v
    


In [ ]:
# https://en.m.wikipedia.org/wiki/Great-circle_distance
# Before loop:
# Fix a point on the unit-sphere.
fixed_point = vecs_on_sphere[:, 0]

delta_sigma = np.empty(shape=vecs_on_sphere.shape[1] - 1)
# Get a vector of greatest-circle
# distances to the other points.
for j in range(1, vecs_on_sphere.shape[1]):
    delta_sigma[j - 1] = np.arctan2(
        np.linalg.norm(np.cross(fixed_point, vecs_on_sphere[:, j]), ord=2),
        np.dot(fixed_point, vecs_on_sphere[:, j])
    )

In [ ]:
# Now, scale to the size of the Earth.
distances = MEAN_RADIUS_OF_EARTH * np.abs(delta_sigma)

In [ ]:
distances.sort()
delta_sigma.sort()

In [ ]:
distances[NUM_WORLD_LOCATIONS - 10: ]

In [ ]:
# https://github.com/SciTools/cartopy/issues/1325
cartopy.config["data_dir"] = Path(DATA_PATH, "raw_data")

In [ ]:
# https://datascience.stackexchange.com/questions/112104/sampling-from-earths-landmass
natural_earth_data = shapereader.natural_earth(
    resolution="10m",
    category="physical",
    name="land"
)

In [ ]:
print(natural_earth_data)

In [ ]:
land_on_earth = shapely.union_all(
    geometries=list(shapereader.Reader(natural_earth_data).geometries())
)

shapely.prepare(
    geometry=land_on_earth
)

In [ ]:
land_on_earth.contains(shapely.geometry.Point(33.792978, -84.412071))

In [ ]:
ecoregions_2017 = geopandas.read_file(filename=Path(DATA_PATH, "raw_data", "Ecoregions2017"))

In [ ]:
ecoregions_2017.crs

In [ ]:
ecoregions_2017.plot()

In [ ]:
ecoregions_2017.head

In [ ]:
# Points have longitude first and latitude second.
ecoregions_2017["geometry"].representative_point()

In [ ]:
ecoregions_2017["geometry"]

In [ ]:
additional_points_on_earth = geopandas.points_from_xy(x=np.array([27.099020]), y=np.array([37.640693]), crs="EPSG:4326")

In [ ]:
ecoregions_2017_geometry = ecoregions_2017["geometry"]

In [ ]:
ecoregions_2017_geometry_union = ecoregions_2017_geometry.unary_union